In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Reshape, Flatten
from scipy.io import loadmat

In [7]:
first100images = loadmat('w135_first100images.mat')['DATAcropped']
print(first100images.shape)

(3320, 399, 100)


In [8]:
first100images = np.transpose(first100images, (2,0,1))
print(first100images.shape)

(100, 3320, 399)


In [9]:
X_train_original = first100images[0:90,:,:].reshape((90, 3320, 399, 1))/(2**16)
X_test_original = first100images[90:100,:,:].reshape((10, 3320, 399, 1))/(2**16)
X_train = np.zeros((90, 3320, 400, 1))
X_test = np.zeros((10, 3320, 400, 1))
X_train[:,:,:399,:] = X_train_original
X_test[:,:,:399,:] = X_test_original

In [10]:
imageshape = X_train.shape[1:4]

In [11]:
autoencoder = Sequential()

# Encoder
autoencoder.add(Conv2D(filters = 32, kernel_size=(3,3), activation='relu', padding='same', input_shape=imageshape))
autoencoder.add(MaxPooling2D(pool_size=(2,2)))

autoencoder.add(Conv2D(filters =16 , kernel_size=(3,3), activation='relu', padding='same'))
autoencoder.add(MaxPooling2D(pool_size=(2,2), padding='same'))

autoencoder.add(Conv2D(filters=8, kernel_size=(3,3), activation='relu', padding = 'same', strides=(2,2)))
autoencoder.add(Flatten())

autoencoder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3320, 400, 32)     320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1660, 200, 32)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1660, 200, 16)     4624      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 830, 100, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 415, 50, 8)        1160      
_________________________________________________________________
flatten (Flatten)            (None, 166000)            0         
Total params: 6,104
Trainable params: 6,104
Non-trainable params: 0
______________________________________________________

2022-07-05 13:45:26.536196: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-05 13:45:26.540995: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.


In [12]:
# Decoder

autoencoder.add(Reshape((415,50,8)))

autoencoder.add(Conv2D(filters = 8, kernel_size=(3,3), activation='relu', padding='same'))
autoencoder.add(UpSampling2D(size=(2,2)))

autoencoder.add(Conv2D(filters = 16, kernel_size=(3,3), activation='relu', padding='same'))
autoencoder.add(UpSampling2D(size=(2,2)))

autoencoder.add(Conv2D(filters = 32, kernel_size=(3,3), activation='relu', padding='same'))
autoencoder.add(UpSampling2D(size=(2,2)))

autoencoder.add(Conv2D(filters = 1, kernel_size=(3,3), activation='sigmoid', padding='same'))
autoencoder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3320, 400, 32)     320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1660, 200, 32)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1660, 200, 16)     4624      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 830, 100, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 415, 50, 8)        1160      
_________________________________________________________________
flatten (Flatten)            (None, 166000)            0         
_________________________________________________________________
reshape (Reshape)            (None, 415, 50, 8)        0

In [13]:
autoencoder.compile(optimizer='Adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [ ]:
autoencoder.fit(X_train, X_train, epochs = 5)

Train on 90 samples
Epoch 1/50
